In [51]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
#from transformers import pipeline

import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import string
import pickle
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

In [52]:
# reading data
df = pd.read_csv(r"D:\hotel_bookings.csv\hotel_bookings.csv")
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [53]:
#dropping columns that are not useful

u= ['days_in_waiting_list', 'arrival_date_year', 'arrival_date_year', 'assigned_room_type', 'booking_changes', 'reservation_status', 'days_in_waiting_list']

df.drop(u, axis = 1, inplace = True)

In [54]:
df.head(10)

,hotel,is_canceled,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,deposit_type,agent,company,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,Resort Hotel,0,342,July,27,1,0,0,2,0.0,...,0,C,No Deposit,NaN,NaN,Transient,0.0,0,0,2015-07-01
1,Resort Hotel,0,737,July,27,1,0,0,2,0.0,...,0,C,No Deposit,NaN,NaN,Transient,0.0,0,0,2015-07-01
2,Resort Hotel,0,7,July,27,1,0,1,1,0.0,...,0,A,No Deposit,NaN,NaN,Transient,75.0,0,0,2015-07-02
3,Resort Hotel,0,13,July,27,1,0,1,1,0.0,...,0,A,No Deposit,304.0,NaN,Transient,75.0,0,0,2015-07-02
4,Resort Hotel,0,14,July,27,1,0,2,2,0.0,...,0,A,No Deposit,240.0,NaN,Transient,98.0,0,1,2015-07-03
5,Resort Hotel,0,14,July,27,1,0,2,2,0.0,...,0,A,No Deposit,240.0,NaN,Transient,98.0,0,1,2015-07-03
6,Resort Hotel,0,0,July,27,1,0,2,2,0.0,...,0,C,No Deposit,NaN,NaN,Transient,107.0,0,0,2015-07-03
7,Resort Hotel,0,9,July,27,1,0,2,2,0.0,...,0,C,No Deposit,303.0,NaN,Transient,103.0,0,1,2015-07-03
8,Resort Hotel,1,85,July,27,1,0,3,2,0.0,...,0,A,No Deposit,240.0,NaN,Transient,82.0,0,1,2015-05-06
9,Resort Hotel,1,75,July,27,1,0,3,2,0.0,...,0,D,No Deposit,15.0,NaN,Transient,105.5,0,0,2015-04-22


In [55]:
# creating numerical and categorical dataframes

x = [col for col in df.columns if df[col].dtype == 'O']
new_df = df[x]
new_df.head()

,hotel,arrival_date_month,meal,country,market_segment,distribution_channel,reserved_room_type,deposit_type,customer_type,reservation_status_date
0,Resort Hotel,July,BB,PRT,Direct,Direct,C,No Deposit,Transient,2015-07-01
1,Resort Hotel,July,BB,PRT,Direct,Direct,C,No Deposit,Transient,2015-07-01
2,Resort Hotel,July,BB,GBR,Direct,Direct,A,No Deposit,Transient,2015-07-02
3,Resort Hotel,July,BB,GBR,Corporate,Corporate,A,No Deposit,Transient,2015-07-02
4,Resort Hotel,July,BB,GBR,Online TA,TA/TO,A,No Deposit,Transient,2015-07-03


In [56]:
new_df['reservation_status_date'] = pd.to_datetime(new_df['reservation_status_date'])

new_df['year'] = new_df['reservation_status_date'].dt.year
new_df['month'] = new_df['reservation_status_date'].dt.month
new_df['day'] = new_df['reservation_status_date'].dt.day
new_df.drop(['reservation_status_date','arrival_date_month'] , axis = 1, inplace = True)


In [57]:
# printing unique values of each column
for c in new_df.columns:
    print(f"{c}: \n{new_df[c].unique()}\n")

hotel: 
['Resort Hotel' 'City Hotel']

meal: 
['BB' 'FB' 'HB' 'SC' 'Undefined']

country: 
['PRT' 'GBR' 'USA' 'ESP' 'IRL' 'FRA' nan 'ROU' 'NOR' 'OMN' 'ARG' 'POL'
 'DEU' 'BEL' 'CHE' 'CN' 'GRC' 'ITA' 'NLD' 'DNK' 'RUS' 'SWE' 'AUS' 'EST'
 'CZE' 'BRA' 'FIN' 'MOZ' 'BWA' 'LUX' 'SVN' 'ALB' 'IND' 'CHN' 'MEX' 'MAR'
 'UKR' 'SMR' 'LVA' 'PRI' 'SRB' 'CHL' 'AUT' 'BLR' 'LTU' 'TUR' 'ZAF' 'AGO'
 'ISR' 'CYM' 'ZMB' 'CPV' 'ZWE' 'DZA' 'KOR' 'CRI' 'HUN' 'ARE' 'TUN' 'JAM'
 'HRV' 'HKG' 'IRN' 'GEO' 'AND' 'GIB' 'URY' 'JEY' 'CAF' 'CYP' 'COL' 'GGY'
 'KWT' 'NGA' 'MDV' 'VEN' 'SVK' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL' 'SEN'
 'SYC' 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MKD' 'MYS' 'ARM' 'JPN' 'LKA' 'CUB'
 'CMR' 'BIH' 'MUS' 'COM' 'SUR' 'UGA' 'BGR' 'CIV' 'JOR' 'SYR' 'SGP' 'BDI'
 'SAU' 'VNM' 'PLW' 'QAT' 'EGY' 'PER' 'MLT' 'MWI' 'ECU' 'MDG' 'ISL' 'UZB'
 'NPL' 'BHS' 'MAC' 'TGO' 'TWN' 'DJI' 'STP' 'KNA' 'ETH' 'IRQ' 'HND' 'RWA'
 'KHM' 'MCO' 'BGD' 'IMN' 'TJK' 'NIC' 'BEN' 'VGB' 'TZA' 'GAB' 'GHA' 'TMP'
 'GLP' 'KEN' 'LIE' 'GNB' 'MNE' 'UMI'

In [58]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [59]:
# encoding categorical variables

new_df['hotel'] = new_df['hotel'].map({'Resort Hotel' : 0, 'City Hotel' : 1})

new_df['meal'] = new_df['meal'].map({'BB' : 0, 'FB': 1, 'HB': 2, 'SC': 3, 'Undefined': 4})
 
new_df['country']=le.fit_transform(new_df['country'])

new_df['market_segment'] = new_df['market_segment'].map({'Direct': 0, 'Corporate': 1, 'Online TA': 2, 'Offline TA/TO': 3, 'Complementary': 4, 'Groups': 5, 'Undefined': 6, 'Aviation': 7})

new_df['distribution_channel'] = new_df['distribution_channel'].map({'Direct': 0, 'Corporate': 1, 'TA/TO': 2, 'Undefined': 3, 'GDS': 4})

new_df['reserved_room_type'] = new_df['reserved_room_type'].map({'C': 0, 'A': 1, 'D': 2, 'E': 3, 'G': 4, 'F': 5, 'H': 6, 'L': 7, 'B': 8})

new_df['deposit_type'] = new_df['deposit_type'].map({'No Deposit': 0, 'Refundable': 1, 'Non Refund': 3})

new_df['customer_type'] = new_df['customer_type'].map({'Transient': 0, 'Contract': 1, 'Transient-Party': 2, 'Group': 3})

new_df['year'] = new_df['year'].map({2015: 0, 2014: 1, 2016: 2, 2017: 3})
new_df['country']

0         135
1         135
2          59
3          59
4          59
         ... 
119385     15
119386     56
119387     43
119388     59
119389     43
Name: country, Length: 119390, dtype: int32

In [60]:
sec_df = df.drop(columns = x, axis = 1)
sec_df['adr'] = sec_df['adr'].fillna(value = sec_df['adr'].mean())

In [61]:
sec_df.head()

,is_canceled,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,agent,company,adr,required_car_parking_spaces,total_of_special_requests
0,0,342,27,1,0,0,2,0.0,0,0,0,0,NaN,NaN,0.0,0,0
1,0,737,27,1,0,0,2,0.0,0,0,0,0,NaN,NaN,0.0,0,0
2,0,7,27,1,0,1,1,0.0,0,0,0,0,NaN,NaN,75.0,0,0
3,0,13,27,1,0,1,1,0.0,0,0,0,0,304.0,NaN,75.0,0,0
4,0,14,27,1,0,2,2,0.0,0,0,0,0,240.0,NaN,98.0,0,1


In [62]:
X = pd.concat([new_df, sec_df], axis = 1)
y = new_df['country']
X=X[['reserved_room_type']]
type(X['reserved_room_type'][0])


numpy.float64

In [63]:
# splitting data into training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train['reserved_room_type'][0]

0.0

In [64]:
X_train.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)
y_train.fillna(0,inplace=True)
y_test.fillna(0,inplace=True)


In [65]:
X_train=X_train.values.reshape(-1,1)
X_test=X_test.values.reshape(-1,1)
y_train=y_train.values.reshape(-1,1)
y_test=y_test.values.reshape(-1,1)

In [66]:
#tfidf=TfidfVectorizer()
clf2=KNeighborsClassifier(n_neighbors=7)
#model =Pipeline([('vectorizer',tfidf),('classifier',clf2)])
clf2.fit(X_train,y_train)
clf2.predict(X_test)

array([135, 135, 135, ..., 135,  51, 135])

# DECISION TREE CLASSIFIER

In [ ]:
Pkl_Filename = "modell.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)    
with open(Pkl_Filename, 'rb') as file:  
    Pickled_Models = pickle.load(file)

In [ ]:
def result(data): 
    data=[data]
    with open('modell.pkl', 'rb') as file:  
        Pickled_Models = pickle.load(file)
        Ypredict = Pickled_Models.predict(data)  
        return Ypredict[0]

In [ ]:
test_string=1.0
result(test_string)

In [66]:
pickle.dump(d,open('model.pkl','wb'))
model=pickle.load(open('model.pkl','rb'))

In [64]:
def functt(inputt):
    l=[]
    l.append(inputt)
    with open('model.pkl','rb') as file:
        y=pickle.load(file)
        yprediction=y.predict(inputt)
        return yprediction[0]
z=1.0
functt(z)

# END

In [ ]:
Pkl_Filename = "modell.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)    
with open(Pkl_Filename, 'rb') as file:  
    Pickled_Models = pickle.load(file)

In [ ]:
def result(data): 
    data=[data]
    with open('modell.pkl', 'rb') as file:  
        Pickled_Models = pickle.load(file)
        Ypredict = Pickled_Models.predict(data)  
        return Ypredict[0]


In [ ]:
test_string='direct'
result(test_string)